# Model Evaluation

### Setup

In [ ]:
import os

from sklearn.metrics import average_precision_score as avp
from sklearn.metrics import precision_recall_curve as prc
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import precision_score, confusion_matrix
from sklearn.utils import shuffle
import numpy as np
from numpy.random import default_rng
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import melanoma_classification.melanomaclassification as mc

In [ ]:
# make sure, the directory for storing results exists.
os.makedirs('../results', exist_ok=True)

In [ ]:
# assign more concise names to the two test-dataframes
isic2016 = mc.isic2016test_std_df
mclass = mc.mclass_std_df

In [ ]:
# assert that there are only two labels
assert(len(isic2016['category'].unique()) == 2)
assert(len(mclass['category'].unique() == 2))

# data generators
isic2016_gen = mc.resnet50_testDataGen_flow_df(
    isic2016,
    mc.images_base_path,
    batch_size=1,
    class_mode='binary')

mclass_gen = mc.resnet50_testDataGen_flow_df(
    mclass,
    mc.images_base_path,
    batch_size=1,
    class_mode='binary')


### Load the models

In [ ]:
twoLayer = tf.keras.models.load_model(
    '../models/twoLayer.h5', compile=False)
simple = tf.keras.models.load_model(
    '../models/simple.h5', compile=False)

In [ ]:
twoLayer.summary()

### Predict using the models and save results

The cells in this section don't have to be run if the results already exist.

In [ ]:
twoLayer_isic2016_predictions = twoLayer.predict_generator(isic2016_gen)
simple_isic2016_predictions = simple.predict_generator(isic2016_gen)
# in this case classes returns an array of 0 and 1. 1 for each malignant
# image and 0 for each benign image.
isic2016['y_true'] = isic2016_gen.classes 
isic2016['y_score_two_Layer'] = twoLayer_isic2016_predictions
isic2016['y_score_simple'] = simple_isic2016_predictions
isic2016.to_csv('../results/isic2016_predictions.csv')
#isic2016

In [ ]:
twoLayer_mclass_predictions = twoLayer.predict_generator(mclass_gen)
simple_mclass_predictions = simple.predict_generator(mclass_gen)
mclass['y_true'] = mclass_gen.classes
mclass['y_score_two_Layer'] = twoLayer_mclass_predictions
mclass['y_score_simple'] = simple_mclass_predictions
mclass.to_csv('../results/mclass_predictions.csv')
#mclass

## Evaluation

### Load results

After executing the first cell, this can be an alternative entry point. 
If the results already exist, there is no need to predict again.

In [ ]:
isic2016 = pd.read_csv('../results/isic2016_predictions.csv')
mclass = pd.read_csv('../results/mclass_predictions.csv')

In [ ]:
# random-classifier-model:
# chooses y_score randomly from a uniform distribution on [0,1].
rg = default_rng(123456789)
isic_random_y_scores = rg.random(isic2016.y_true.values.shape)
mclass_random_y_scores = rg.random(mclass.y_true.values.shape)

### Evaluation Function

In [ ]:
def evaluate_model(y_true, y_score, model_name='NoName'):
    '''
    y_true and y_score shall be 1-D numpy ndarrays
    '''
    results_dict = {}
    results_dict['model_name'] = model_name
    results_dict['average_precision'] = avp(y_true, y_score)
    precision, recall, prc_thresholds = prc(y_true, y_score)
    results_dict['precision'] = np.flip(precision)
    results_dict['recall'] = np.flip(recall)
    results_dict['prc_thresholds'] = np.flip(prc_thresholds)
    
    results_dict['roc_auc'] = roc_auc_score(y_true, y_score)
    fpr, tpr, roc_thresholds = roc_curve(y_true, y_score)
    results_dict['fpr'] = fpr
    results_dict['tpr'] = tpr
    results_dict['roc_thresholds'] = roc_thresholds
    
    return results_dict

In [ ]:
# isic
isic_tL_eval = evaluate_model(isic2016.y_true.values, 
                                              isic2016.y_score_two_Layer, 
                                              model_name='twoLayer')
isic_tL_eval['color'] = 'b'

isic_simple_eval = evaluate_model(isic2016.y_true.values, 
                                              isic2016.y_score_simple, 
                                              model_name='simple')

isic_simple_eval['color'] = 'r'
isic_random_eval = evaluate_model(isic2016.y_true.values,
                                  isic_random_y_scores,
                                  model_name='random')

isic_random_eval['color'] = 'g'


# mclass
mclass_tL_eval = evaluate_model(mclass.y_true.values, 
                                              mclass.y_score_two_Layer, 
                                              model_name='twoLayer')
mclass_tL_eval['color'] = 'b'

mclass_simple_eval = evaluate_model(mclass.y_true.values, 
                                              mclass.y_score_simple, 
                                              model_name='simple')

mclass_simple_eval['color'] = 'r'
mclass_random_eval = evaluate_model(mclass.y_true.values,
                                  mclass_random_y_scores,
                                  model_name='random')

mclass_random_eval['color'] = 'g'



In [ ]:
print('ISIC2016\n-------')
for model in (isic_tL_eval, isic_simple_eval, isic_random_eval):
    print('''Model: {},
  Average Precision = {:.4f},
  ROC-AUC = {:.4f}'''.format(
       model['model_name'], model['average_precision'], model['roc_auc']))
    model['dataset'] = 'isic2016'

print('\n\n')
    
print('MClass\n------')
for model in (mclass_tL_eval, mclass_simple_eval, mclass_random_eval):
    print('''Model: {},
  Average Precision = {:.4f},
  ROC-AUC = {:.4f}'''.format(
       model['model_name'], model['average_precision'], model['roc_auc']))
    model['dataset'] = 'MClass'

In [ ]:
eval_df = pd.DataFrame(
    [isic_tL_eval, isic_simple_eval, isic_random_eval, 
     mclass_tL_eval, mclass_simple_eval, mclass_random_eval], 
    columns=['model_name', 'dataset', 'average_precision', 'roc_auc'])
eval_df.set_index(['dataset', 'model_name'], inplace=True, 
                  verify_integrity=True)
# eval_df

In [ ]:
d_eval = eval_df.copy()
d_eval.rename(columns={'average_precision': 'AveP', 'roc_auc': 'AUC'}, 
              inplace=True)
d_eval.index.names = ['Dataset', 'Model']
# d_eval

In [ ]:
# print LaTeX format
print(d_eval.to_latex(float_format="{:0.4f}".format))

### Plots

In [ ]:
def plot_diagram(fig, ax, model, xkey, ykey, 
                 xlabel='', ylabel='', title='', style='.-'):
    ax.plot(model[xkey], model[ykey], style, 
            label=model['model_name'], color=model['color'], 
           linewidth = 0.5)
    ax.fill_between(model[xkey], model[ykey]
                    , color=model['color'], alpha=0.1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_aspect('equal')
    
    return fig, ax

In [ ]:
def plot_precision_recall_curve(models, title='',fig=None, 
                                ax=None, style='.-'):
    if None in (fig, ax):
        fig, ax = plt.subplots()
    for model in models:
        plot_diagram(fig, ax, model, 'recall', 'precision', 
                     xlabel='Recall', ylabel='Precision', 
                     title=title, style=style)

    ax.grid(linestyle='-', linewidth=0.1)
    ax.legend()
    return fig,ax

def plot_roc_curve(models, title='',fig=None, ax=None, style='.-'):
    if None in (fig, ax):
        fig, ax = plt.subplots()
    for model in models:
        plot_diagram(fig, ax, model, 'fpr', 'tpr', 
                     xlabel='FPR', ylabel='TPR', 
                     title=title, style=style)

    ax.grid(linestyle='-', linewidth=0.1)
    ax.legend()
    return fig,ax

In [ ]:
# To uncomment, if you want subplots.
# fig, ax_arr = plt.subplots(nrows=1, ncols=2, figsize=(15,7.5))
# fig, ax_arr[0] = plot_precision_recall_curve(
#     (isic_tL_eval, 
#      isic_simple_eval, 
#      isic_random_eval), 
#     'Precision-Recall-Diagramm - ISIC 2016',
#     fig=fig, ax=ax_arr[0],
# )
# ax_arr[0].set_aspect('equal')
# #ax_arr[0].set_xlim(0,1)
# #ax_arr[0].set_ylim(0,1)

# fig, ax_arr[1] = plot_roc_curve(
#     (isic_tL_eval, 
#      isic_simple_eval, 
#      isic_random_eval), 
#     'ROC Analyse - ISIC 2016',
#     fig=fig, ax=ax_arr[1],
# )
# ax_arr[1].set_aspect('equal')


# fig.savefig(
#     '../results/ISIC2016_subplots.pdf',
#     bbox_inches='tight',
#     pad_inches=0,
# )

In [ ]:
fig1, ax1 = plot_precision_recall_curve(
    (isic_tL_eval, 
     isic_simple_eval, 
     isic_random_eval), 
    'Precision-Recall - ISIC 2016',
)
fig1.savefig(
    '../results/prc-ISIC2016.pdf',
    bbox_inches='tight',
    pad_inches=0,
)

In [ ]:
fig2, ax2 = plot_roc_curve(
    (isic_tL_eval, 
     isic_simple_eval, 
     isic_random_eval), 
    'ROC Analysis - ISIC 2016')
fig2.savefig(
    '../results/roc-ISIC2016.pdf',
    bbox_inches='tight',
    pad_inches=0,
)




In [ ]:
fig3, ax3 = plot_precision_recall_curve(
    (mclass_tL_eval, 
     mclass_simple_eval, 
     mclass_random_eval), 
    'Precision-Recall - MClass')
fig3.savefig(
    '../results/prc-mclass.pdf',
    bbox_inches='tight',
    pad_inches=0,
)


In [ ]:
fig4, ax4 = plot_roc_curve(
    (mclass_tL_eval, 
     mclass_simple_eval, 
     mclass_random_eval), 
    'ROC Analysis  - MClass')
fig4.savefig(
    '../results/roc-mclass.pdf',
    bbox_inches='tight',
    pad_inches=0,
)

## Comparison to Dermatologists

In [ ]:
mclass_dermatologist_results = pd.read_csv(
    '../metadata/MClass_ResultsDermoscopic.csv', index_col='Dermatologist')
mclass_dermatologist_results.replace(
    to_replace='biopsy / further treatment', value=1, inplace=True)
mclass_dermatologist_results.replace(
    to_replace='no further treatment', value=0, inplace=True)
mclass_dermatologist_results.rename(
    inplace=True,
    columns={s:i for s,i in zip(
        mclass_dermatologist_results.columns, 
        range(0, len(mclass_dermatologist_results.columns)))})
mclass_dermatologist_results = mclass_dermatologist_results.transpose()
mclass_dermatologist_results
mclass_dermatologist_results.index.name = 'id'
mclass_dermatologist_results.rename(
    inplace=True,
    index=dict(mclass['id'])
)
# mclass_dermatologist_results

In [ ]:
def mclass_recall(y_pred):
    return recall_score(mclass['y_true'], y_pred)

dermatologist_recall = mclass_dermatologist_results.apply(mclass_recall, 
                                                          axis=0)

def mclass_precision(y_pred):
    return precision_score(mclass['y_true'], y_pred)

def mclass_fpr(y_pred):
    conf_matrix = confusion_matrix(mclass['y_true'], y_pred)
    tn, fp, fn, tp = confusion_matrix(mclass['y_true'], y_pred).ravel()

    fpr = fp / (fp+tn)
    return fpr

dermatologist_recall = mclass_dermatologist_results.apply(
    mclass_recall, axis=0)
dermatologist_precision = mclass_dermatologist_results.apply(
    mclass_precision, axis=0)

dermatologist_fpr = mclass_dermatologist_results.apply(
    mclass_fpr, axis = 0)

dermatologist_evaluation = pd.DataFrame(data={
    'recall': dermatologist_recall,
    'precision': dermatologist_precision,
    'fpr': dermatologist_fpr})
#dermatologist_evaluation


In [ ]:
fig5, ax5 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
fig5, ax5 = plot_precision_recall_curve(
    (mclass_tL_eval,), 
    'Precision-Recall - MClass, Dermatologists',style='-',
    fig=fig5, ax=ax5,
)
ax5.plot(
    dermatologist_recall, 
    dermatologist_precision, '.g', label='Dermatologists')
ax5.legend()
fig5.savefig(
    '../results/prc-mclass-dermatologists.pdf',
    bbox_inches='tight',
    pad_inches=0,
)

In [ ]:
fig6, ax6 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))

fig6, ax6 = plot_roc_curve(
    (mclass_tL_eval,), 
    'ROC Analyse - MClass, Dermatologists',style='-',
    fig=fig6, ax=ax6,
)
ax6.plot(
    dermatologist_fpr, 
    dermatologist_recall, '.g', label='Dermatologists')
ax6.legend(loc='lower right')
fig6.savefig(
    '../results/roc-mclass-dermatologists.pdf',
    bbox_inches='tight',
    pad_inches=0,
)

### Box Plot 

In [ ]:
tL_probas_mal = mclass['y_score_two_Layer'][mclass['y_true']==1]
tL_probas_ben = mclass['y_score_two_Layer'][mclass['y_true']==0]

fig7, ax7 = plt.subplots()
ax7.boxplot([tL_probas_mal, tL_probas_ben], labels =['malignant', 'benign'], 
                #showfliers=False,
                #whis=1.2
)
ax7.set_ylabel('Model-Score')
ax7.set_title('MClass: twoLayer Model scores for each category')
fig7.savefig('../results/boxplot.pdf',
            bbox_inches='tight',
             pad_inches=0,
            )

### Image examples

Show images of the program excelling and failing.

In [ ]:
# mclass

In [ ]:
mclass[mclass['y_true']==0].sort_values(
    by=['y_score_two_Layer']).head()#['id.1']

In [ ]:

fig8, ax8 = plt.subplots(nrows = 4, ncols=5, 
                         gridspec_kw={'hspace': 0, 'wspace': 0}, 
                         figsize=(5, 4))

for ax, fname in zip(
        ax8[0], 
        mclass[mclass['y_true']==1].sort_values(
            by=['y_score_two_Layer']).tail(5)['id.1']):
    ax.axis('Off')
    img = mpimg.imread('../Images/' + fname)
    imgplot = ax.imshow(img,)
    
    
    
for ax, fname in zip(
        ax8[1], 
        mclass[mclass['y_true']==1].sort_values(
            by=['y_score_two_Layer']).head(5)['id.1']):
    ax.axis('Off')
    img = mpimg.imread('../Images/' + fname)
    imgplot = ax.imshow(img,)
    
for ax, fname in zip(
        ax8[2], 
        mclass[mclass['y_true']==0].sort_values(
            by=['y_score_two_Layer']).head(5)['id.1']):
    ax.axis('Off')
    img = mpimg.imread('../Images/' + fname)
    imgplot = ax.imshow(img,)
    
for ax, fname in zip(
        ax8[3], 
        mclass[mclass['y_true']==0].sort_values(
            by=['y_score_two_Layer']).tail(5)['id.1']):
    ax.axis('Off')
    img = mpimg.imread('../Images/' + fname)
    imgplot = ax.imshow(img,)
    
    
fig8.subplots_adjust(bottom=0, left=0, right=1, top=1)
    
    
fig8.savefig('../results/images_rows.pdf', dpi=300, 
             bbox_inches='tight',
             pad_inches=0,
            )


### Count points inside polygon

#### precision - recall 

In [ ]:
mclass_tL_precision = mclass_tL_eval['precision']
mclass_tL_recall = mclass_tL_eval['recall']

In [ ]:
mclass_tL_precision_polygon = np.append(mclass_tL_precision, [0, 0])
mclass_tL_recall_polygon = np.append(mclass_tL_recall, [1, 0])
# mclass_tL_precision_polygon
# mclass_tL_recall_polygon

In [ ]:
prc_polyg = Polygon([(x,y) for (x,y) in zip(
    mclass_tL_recall_polygon, mclass_tL_precision_polygon)])
# prc_polyg

In [ ]:
prc_points_list = [Point(x,y) for x, y in zip(
    dermatologist_recall, dermatologist_precision)]
# prc_points_list[5]

In [ ]:
prc_inside = pd.Series([prc_polyg.contains(p) for p in prc_points_list])
prc_touch = pd.Series([prc_polyg.touches(p) for p in prc_points_list])
prc_disjoint = pd.Series([prc_polyg.disjoint(p) for p in prc_points_list])

In [ ]:
# prc_inside.value_counts()

In [ ]:
# prc_touch.value_counts()

In [ ]:
# prc_disjoint.value_counts()

In [ ]:
mclass_tL_ = mclass_tL_eval['precision']
mclass_tL_recall = mclass_tL_eval['recall']

In [ ]:
mclass_tL_fpr = mclass_tL_eval['fpr']
mclass_tL_tpr = mclass_tL_eval['tpr']
# mclass_tL_fpr
# mclass_tL_tpr

In [ ]:
mclass_tL_fpr_polygon = np.append(mclass_tL_fpr, [1])
mclass_tL_tpr_polygon = np.append(mclass_tL_tpr, [0])
# mclass_tL_fpr_polygon
# mclass_tL_tpr_polygon

In [ ]:
roc_polyg = Polygon([(x,y) for (x,y) in zip(
    mclass_tL_fpr_polygon, mclass_tL_tpr_polygon)])
# roc_polyg

In [ ]:
roc_points_list = [Point(x,y) for x, y in zip(
    dermatologist_fpr, dermatologist_recall)]
# roc_points_list[5]

In [ ]:
roc_inside = pd.Series([roc_polyg.contains(p) for p in roc_points_list])
roc_touch = pd.Series([roc_polyg.touches(p) for p in roc_points_list])
roc_disjoint = pd.Series([roc_polyg.disjoint(p) for p in roc_points_list])

In [ ]:
# roc_inside.value_counts()

In [ ]:
# roc_touch.value_counts()

In [ ]:
# roc_disjoint.value_counts()

In [ ]:
points_eval_df = pd.DataFrame(
    {
        'd_fpr': list(dermatologist_fpr),
        'd_recall': list(dermatologist_recall),
        'd_precision': list(dermatologist_precision),
        'prc_inside': prc_inside,
        'prc_touch': prc_touch,
        'prc_disjoint': prc_disjoint,
        'roc_inside': roc_inside,
        'roc_touch': roc_touch,
        'roc_disjoint': roc_disjoint,
    
    })
# points_eval_df

In [ ]:
roc_points_eval = pd.Series(
    {key:val.value_counts()[True] for (key,val) in zip(
        ('below', 'on', 'above'), (roc_inside, roc_touch, roc_disjoint))})
prc_points_eval = pd.Series(
    {key:val.value_counts()[True] for (key,val) in zip(
        ('below', 'on', 'above'), (prc_inside, prc_touch, prc_disjoint))})
points_df = pd.DataFrame({'PRC': prc_points_eval, 'ROC':roc_points_eval})

# roc_points_eval
# prc_points_eval
# points_df
print(points_df.to_latex())

### pictures from training set

In [ ]:
ts = mc.training_set
# ts

In [ ]:
ts_malignant = ts[ts['category']=='malignant']
malignant_ts = shuffle(ts_malignant, random_state=1).head(5)
# malignant_ts

In [ ]:
ts_benign = ts[ts['category']=='benign']
benign_ts = shuffle(ts_benign, random_state=10).head(5)
# benign_ts

In [ ]:
fig9, ax9 = plt.subplots(nrows = 2, ncols=5, 
                         gridspec_kw={'hspace': 0, 'wspace': 0}, 
                         figsize=(5, 2))

for ax, fname in zip(
        ax9[0], 
        malignant_ts['id']):
    ax.axis('Off')
    img = mpimg.imread('../Images/' + fname)
    imgplot = ax.imshow(img,)
    
    
    
for ax, fname in zip(
        ax9[1], 
        benign_ts['id']):
    ax.axis('Off')
    img = mpimg.imread('../Images/' + fname)
    imgplot = ax.imshow(img,)
    
   
    
fig9.subplots_adjust(bottom=0, left=0, right=1, top=1)
    
    
fig9.savefig('../results/images_training_set.pdf', dpi=300, 
             bbox_inches='tight',
             pad_inches=0,
            )

### random: Explanation of the Precision-Recall curves


In [ ]:
isic2016['y_score_random'] = isic_random_y_scores
random_sorted = list(isic2016.sort_values(
    by='y_score_random')['category'].replace(
    to_replace={'benign':0, 'malignant':1}))

In [ ]:
random_sorted[-10:]